In [1]:
using TaylorSeries, TaylorIntegration
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 20
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-20
const steps = 500_000

500000

In [2]:
function undrivenHelium!(τ, q, dq)
    tT, Q₁, Q₂, P₁, P₂ = q
    
    t1 = Q₁^2
    t2 = Q₂^2
    t = t1 * t2
    dq[1] = t
    R12 = t1 - t2
    R122 = R12^2
    RRR = (R122)^(3/2)
   
    dq[2] = 0.25*t2*P₁
    dq[4] = 2Q₁*(-0.125P₂^2 + Z  - t2 - t2/sqrt(R122) + t*R12/RRR)
    dq[3] = 0.25*t1*P₂
    dq[5] = 2Q₂*(-0.125P₁^2 + Z  - t1 - t1/sqrt(R122) - t*R12/RRR)

#    return [t, q₁, q₂, p₁, p₂] 
    nothing
end
#undrivenHelium(v) = undrivenHelium(zero(eltype(v)), v)


var1 = set_variables("t q1 q2 p1 p2", order = 1)


function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [0, Q1, Q2, P1, P2]
end

function regHam(t, Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs(Q₁² - Q₂²)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

function errHam(N::Int)
    srand(487293456)
    J = [0 0 0 0 0; 0 0 0 1 0; 0 0 0 0 1; 0 -1 0 0 0; 0 0 -1 0 0 ]
    
    dnorm = zeros(N)
    als = typeof(zeros(5))[]
    for j in 1:N
        al = 10*rand(5)
        #al = [BigFloat(x) for x in al1]
        alt = al + var1
        ene = J*∇(regHam(alt))
        ene1 = [ene[k].coeffs[1].coeffs[1] for k in 1:5]
        meq = undrivenHelium(al)
        push!(als, al)
        meq[1] = 0
        dnorm[j] = norm(meq - ene1) 
    end
    return dnorm, als 
end

errHam (generic function with 1 method)

In [3]:
function q2sign(xold, x0, tol)
    if abs(xold[3]) < tol
        sq2o = 0
    else
        sq2o = sign(xold[3])
    end
    
    if abs(x0[3]) < tol
        sq2 = 0
    else
        sq2 = sign(x0[3])
    end
    if abs(xold[5]) < tol
        sp2o = 0
    else
        sp2o = sign(xold[5])
    end
    
    if abs(x0[5]) < tol
        sp2 = 0
    else
        sp2 = sign(x0[5])
    end
    return sq2o, sq2, sp2o, sp2
end
    

q2sign (generic function with 1 method)

In [4]:
a1 = condini(rand(2)...)

5-element Array{Float64,1}:
 0.0     
 0.344379
 0.0     
 0.129793
 4.0     

In [10]:
b1 = a1+var1

5-element Array{TaylorSeries.TaylorN{Float64},1}:
                         1.0 t + 𝒪(‖x‖²)
    0.344379282501321 + 1.0 q1 + 𝒪(‖x‖²)
                        1.0 q2 + 𝒪(‖x‖²)
  0.12979340486329646 + 1.0 p1 + 𝒪(‖x‖²)
                  4.0 + 1.0 p2 + 𝒪(‖x‖²)

In [11]:
x = Array{Taylor1{TaylorN{Float64}}}(5)

5-element Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [13]:
for i in eachindex(b1)
        @inbounds x[i] = Taylor1( b1[i], order )
    end

In [18]:
ord = x[1].order

20

In [15]:
xaux = Array{Taylor1{TaylorN{Float64}}}(5)

5-element Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [16]:
dx = Array{Taylor1{TaylorN{Float64}}}(5)

5-element Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [25]:
x[1].coeffs[1:ord] 

20-element Array{TaylorSeries.TaylorN{Float64},1}:
  1.0 t + 𝒪(‖x‖²)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)
    0.0 + 𝒪(‖x‖¹)

In [19]:
        for j in eachindex(x)
            @inbounds xaux[j] = Taylor1( x[j].coeffs[1:ord] )
        end


In [20]:
xaux

5-element Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}:
                          1.0 t + 𝒪(‖x‖²) + 𝒪(t²⁰)
     0.344379282501321 + 1.0 q1 + 𝒪(‖x‖²) + 𝒪(t²⁰)
                         1.0 q2 + 𝒪(‖x‖²) + 𝒪(t²⁰)
   0.12979340486329646 + 1.0 p1 + 𝒪(‖x‖²) + 𝒪(t²⁰)
                   4.0 + 1.0 p2 + 𝒪(‖x‖²) + 𝒪(t²⁰)

In [21]:
undrivenHelium!(0.0, xaux, dx)

LoadError: InterruptException:

In [24]:
xaux*xaux

LoadError: MethodError: no method matching *(::Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}, ::Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1})[0m
Closest candidates are:
  *(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  *([1m[31m::PyCall.PyObject[0m, ::Any) at /home/daniel/.julia/v0.5/PyCall/src/PyCall.jl:702
  *{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S}([1m[31m::Union{Base.ReshapedArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N}}},DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:Union{Base.ReshapedArray{T,N,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{In

In [ ]:
dx

In [ ]:
dx

In [ ]:
for j in eachindex(x)
            @inbounds x[j].coeffs[2] = dx[j].coeffs[1]/1
        end

In [ ]:
taylorinteg(undrivenHelium!, a1, 0.0, 10.0, order, abs_tol, maxsteps=5)

In [ ]:
taylorinteg(undrivenHelium!, x, 0.0, 10.0, order, abs_tol, maxsteps=1)

In [ ]:
taylorinteg()

In [ ]:
function vareqs(t, q, A)
    qn = A * q
    return qn
end

stabmat = eye(5)
stabmat[1, 1] = 0.0

varuh(t, q) = vareqs(t, q, stabmat)

In [ ]:
a1 = condini(rand(2)...)
expe1 = var1 + a1

In [ ]:
varuh(0.0, var1)

In [ ]:
var1[2].coeffs[2].coeffs

In [ ]:
typeof(var1)

In [ ]:
###Implementamos nuevo código 
function taylorstep1!{T<:Number}(f, t0::T, t1::T, x0::Array{T,1},
        order::Int, abstol::T)
    @assert t1 > t0
    # Initialize the vector of Taylor1 expansions
    xT = Array{Taylor1{TaylorN{T}}}(length(x0))
    for i in eachindex(x0)
        @inbounds xT[i] = Taylor1( x0[i], order )
    end
    # Compute the Taylor coefficients
    jetcoeffs!(f, t0, xT)
    # Compute the step-size of the integration using `abstol`
    δt = TaylorIntegration.stepsize(xT, abstol)
    if δt ≥ t1-t0
        δt = t1-t0
    end
    evaluate!(xT, δt, x0)
    return δt, xT
end


function mytaylorinteg{T<:Number}(f, q0::Array{TaylorN{T},1}, t0::T, tmax::T, order::Int,
    abstol::T; tsteps::Int = 1000, tol::Float64=1e-20, maxsteps::Int=5000)

    # Allocation
    tv = Array{T}(maxsteps)
    dof = length(q0)
    xv = Array{T}(dof, maxsteps)
    δs = Array{TaylorN}(maxsteps)
    sm0 = eye(dof)
    
    function vareqs(t, q, A)
        qn = A * q
        return qn
    end
    varuh(t, q) = vareqs(t, q, sm0)
    
    
    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] = q0[:]
    @inbounds δs[1] = var1
    x0 = copy(q0)
    δ0 = copy(δs[1])
    sum1 = q0[3]/maxsteps

    # Integration
    nsteps = 1
    δtn = Inf
    while t0 < tmax
        xold = copy(x0)
        δold = copy(δ0)
        δt, sT = taylorstep1!(f, t0, tmax, x0, δ0, order, abstol)
        sq2o, sq2, sp2o, sp2 = q2sign(xold, x0, tol)
        steps1 = 0
        x00 = copy(x0)
        δ00 = copy(δ0)
        bool1 = false
        
        while sq2o*sq2 == -1 || (sp2o*sp2 == -1 && abs(xold[3]) < tol)
            bool1 = true
            q2T = sT[3]
            dq2T = derivative(q2T)
            δtn = copy(δt)
            for nc in 1:20
                δtn = δtn - evaluate(q2T, δtn)/evaluate(dq2T, δtn)
            end
            evaluate!()
            evaluate!(sT, δtn, x0)
            sq2o, sq2, sp2o, sp2 = q2sign(xold, x0, tol)
            steps1 += 1
            if steps1  ≥ tsteps
                break
            end
        end
                
        if nsteps ≥ maxsteps 
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
        
                
        nsteps += 1
        
        if bool1 == true
            @inbounds tv[nsteps] = t0 + δtn
            @inbounds xv[:,nsteps] = x0[:]
            stabilitymatrix!(undrivenHelium, t0 + δtn, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 1]
            arrjac[nsteps] = jjac0
            x0 = x00
            nsteps += 1
            stabilitymatrix!(undrivenHelium, t0 + δt, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 2]
            arrjac[nsteps] = jjac0
        else
            stabilitymatrix!(undrivenHelium, t0 + δt, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 1]
            arrjac[nsteps] = jjac0            
        end
        t0 += δt
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] = x0[:]
        sum1 += x0[3]/maxsteps
            
    end

    return view(tv,1:nsteps), view(xv,:,1:nsteps)', arrjac[1:nsteps]
end

In [ ]:
c1 = condini(2.0, 0.0) + var1

In [ ]:
@time t1, x1 = taylorinteg(undrivenHelium, c1, t0, 2.0, order, abs_tol, maxsteps=100_000);

In [ ]:
@time t2, x2, m1 = mytaylorinteg(undrivenHelium, c1, t0, 2.0, order, abs_tol, maxsteps=100)

In [ ]:
m1[64]

In [ ]:
for j in 1:64
    a = (m1[j][2:end,2:end])
    println(a)
end